In [57]:
%load_ext autoreload
import utils

from pls_tools import MultitablePLSC as mpls
from corr_tools import cross_corr

import numpy as np

def build_corr_xy(y, x_list):
    num_vars_in_y = y.shape[1]
    num_vars_in_x = [x.shape[1] for x in x_list]

    cross_xy = np.ndarray(shape=[num_vars_in_y, sum(num_vars_in_x)])

    start_index = 0
    for x_index, x_table in enumerate(x_list):
        cross_corrmat = cross_corr(y, x_table)
        end_index = start_index + num_vars_in_x[x_index]
        cross_xy[:, start_index:end_index] = cross_corrmat

    return cross_xy

def procrustes_rotation(orig_svd, resamp_svd):
    ov = orig_svd[0]
    rv = resamp_svd[0]
    rs = resamp_svd[1]
    ru = resamp_svd[2]

    n, _, p = np.linalg.svd(np.matmul(ov.T, rv))

    q = n * p
        
    rot_u = ru * rs * q
    rot_v = rv * rs * q

    ss_rot_v = np.sum(rot_v**2, 0) #sum of squares of rotated v
    rot_s = np.sqrt(ss_rot_v)

    return rot_u, rot_s, rot_v

def p_from_perm_mat(obs_vect, perm_array):
    n_iters = perm_array.shape[0]
    p_values = np.ndarray(shape=obs_vect.shape)
    for t, true in enumerate(obs_vect):
        perm_data = perm_array[t, :]
        p_values[t] = utils.permutation_p(true, perm_data, n_iters)
    return p_values

def mult_plsc(y_table, x_tables):
    corr_xy = build_corr_xy(y_table, x_tables)
    centered_corr_xy = utils.center_matrix(corr_xy)

    u, delta, v = np.linalg.svd(centered_corr_xy, full_matrices=False)
    return u, delta, v.T

def perm_mult_plsc(y_table, x_tables):
    orig_svd = mult_plsc(y_table, x_tables)
    orig_sing = orig_svd[1]

    n = 0
    perm_singular_values = np.ndarray(shape=[1000, len(orig_sing)])
    while n != 1000:
        perm_y = utils.perm_matrix(y_table)
        perm_x_tables = [utils.perm_matrix(x) for x in x_tables]

        perm_svd = mult_plsc(perm_y, perm_x_tables)
        rot_perm = procrustes_rotation(orig_svd, perm_svd)

        perm_singular_values[n, :] = rot_perm[1]

    p_values = utils.p_from_perm_mat(orig_sing, perm_singular_values)

    if self.return_perm:
        return p_values, perm_singular_values
    else:
        return p_values

n = 10
y = np.random.rand(n, 30)

print('Y-table has shape: %s' % str(y.shape))
x_list = []
for numx in range(2):
    x_array = np.random.rand(n, np.random.randint(5, 11))
    x_list.append(x_array)
    print('X-table %d has shape: %s' % (numx, x_array.shape))

corr_xy = build_corr_xy(y, x_list)
print('Cross-correlation stack has shape: %s' % str(corr_xy.shape))

res = np.linalg.svd(corr_xy, full_matrices=False)
u = res[0]
s = res[1]
v = res[2]

print('U has shape: %s' % str(u.shape))
print('Delta has length: %s' % str(s.shape))
print('V has shape: %s' % str(v.T.shape))

original_v = v
perm_u = u
perm_diag = s
perm_v = v

n, _, p = np.linalg.svd(np.dot(original_v.T, perm_v), full_matrices=False)
rotation_matrix = n.dot(p.T)
print('Rotation matrix has shape: %s' % str(rotation_matrix.shape))
rotated_u = np.dot(np.dot(perm_u, np.diagflat(perm_diag)), rotation_matrix)
print('Rotated left singular values has shape: %s' % str(rotated_u.shape))

rotated_v = np.dot(rotation_matrix, np.dot(np.diagflat(perm_diag), perm_v))
print('Rotated right singular values has shape: %s' % str(rotated_v.shape))

print(np.sum(rotated_v[:, :]**2, 0))

# n, _, p = np.linalg.svd(np.dot(u.T, u))
# q = n.dot(p.T)
# print('Rotation matrix has shape: %s' % str(q.shape))

# rot_u = np.dot(np.dot(np.diagflat(s), u), q)
# print('Rotated left singular values has shape: %s' % str(rot_u.shape))

# rot_v = np.dot(q, np.dot(np.diagflat(s), v))
# print('Rotated right singular values has shape: %s' % str(rot_v.shape))
# #print(rot_v)
# rot_v_squared = np.square(rot_v)

# ss_rot_v = np.sum(rot_v*rot_v, 0) #sum of squares of rotated v
# rot_s = np.sqrt(ss_rot_v)


#perm_mult_plsc(y, x_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Y-table has shape: (10, 30)
X-table 0 has shape: (10, 5)
X-table 1 has shape: (10, 7)
Cross-correlation stack has shape: (30, 12)
U has shape: (30, 12)
Delta has length: (12,)
V has shape: (12, 12)
Rotation matrix has shape: (12, 12)
Rotated left singular values has shape: (30, 12)
Rotated right singular values has shape: (12, 12)
[2.96035545 4.85854836 3.2804752  3.52012913 2.18422598 4.11035352
 3.12441054 0.         0.         0.         0.         0.        ]


In [88]:
%autoreload
print('Starting')
# p = mpls(n_iters=1000, return_perm=True)
# res = p.perm_mult_plsc(y, x_list)
n = 5
print(res['true_eigenvalues'][n])
print(res['p_values'][n])
# print(res['permutation_eigs'][:, n])
print('Finished')

n_iters = len(res['permutation_eigs'][:, n])
n_hits = np.where(np.abs(res['permutation_eigs'][:, n]) >= np.abs(res['true_eigenvalues'][n]))

print(len(n_hits[0]))

Starting
1.1580871325774018
0.001998001998001998
Finished
1000


In [87]:
n_hits

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177